In [5]:
import os

print(os.listdir())

['.ipynb_checkpoints', '110_Lecture_Notes_PDF_NEW', '110_Lecture_Notes_PDF_NEW.zip', '12_8_2024.jpg', '2022.txt', '3a2f8242-d7fd-43c3-b7d7-a1bebb116fef.jfif', '47830885281_da079220b1_o.jpg', '503beaea-0443-4609-a7f7-61ecde0ea9f9.jpg', '5ed64246-fded-4289-91c4-48447b2aa7fa.jpg', '6181682361561891577.jpg', '71mxOxm3YiL._AC_UF894,1000_QL80_.jpg', '8576658_Lin_Shiming.pdf', 'A-1-Questions.pdf', 'aaaa.png', 'Agile methodology_XP_SCRUM (1).docx', 'ans', 'ans.zip', 'Applicant declaration (FSS).pdf', 'ATLYSS.url', 'b6b43d7d-ef5d-4153-aa14-9ec9ee6b666d.jpg', 'Beyerdynamic MMX100 ParametricEq.txt', 'clues', 'computing', 'CSIT 123 lecture slides', 'csit115 exam past paper (1).pdf', 'CSIT121_Assignment_3_2024S4.pdf', 'CSIT224', 'Cybersec submission A1', 'default', 'deskmat_sketch_act_size_2.png.kra', 'deskmat_sketch_act_size_3.png.kra', 'deskmat_sketch_act_size_3.png.kra~', 'desktop.ini', 'Disco Elysium.url', 'Discord.lnk', 'doc', 'document', 'documents', 'Download.mp4', 'drawing', 'drivingh', 'Ec